In [ ]:
import torch
import torch.nn as nn
from torch.func import functional_call, grad, vmap
import numpy as np

# Define la arquitectura de la red neuronal
class RedNeuronalLineal(nn.Module):
    def __init__(self, num_entradas=1, num_capas=1, num_neuronas=5, act=nn.Tanh()):
        super().__init__()
        self.num_entradas = num_entradas
        self.num_neuronas = num_neuronas
        self.num_capas = num_capas

        capas = [nn.Linear(self.num_entradas, num_neuronas)]
        for _ in range(num_capas):
            capas.extend([nn.Linear(num_neuronas, num_neuronas), act])
        capas.append(nn.Linear(num_neuronas, 1))

        self.red = nn.Sequential(*capas)

    def forward(self, x):
        return self.red(x.reshape(-1, 1)).squeeze()

# Define la solución analítica para el crecimiento logístico
def crecimiento_logistico(t, R, K, P0):
    return K / (1 + ((K - P0) / P0) * np.exp(-R * t))

# Función para convertir los parámetros del modelo a un diccionario
def obtener_diccionario_params(modelo):
    return {nombre: param for nombre, param in modelo.named_parameters()}

# Inicializa el modelo y obtén los parámetros iniciales como un diccionario
modelo = RedNeuronalLineal(num_capas=3, num_neuronas=10)
params_dict = obtener_diccionario_params(modelo)

# Funciones para calcular gradientes de orden superior
def f(x, params_dict):
    return functional_call(modelo, params_dict, (x,))

dfdx = vmap(grad(f), in_dims=(0, None))

# Parámetros para la función de pérdida y el modelo de crecimiento logístico
R = 1.0  # Parámetro de tasa de crecimiento
K = 1.0  # Capacidad de carga
P0 = 0.5  # Tamaño de la población inicial
X_BOUNDARY = 0.0  # Coordenada de la condición límite
F_BOUNDARY = P0  # Valor de la condición límite

# Función de pérdida
def funcion_perdida(params_dict, x, y_analitica):
    valor_f = f(x, params_dict)
    interior = dfdx(x, params_dict) - R * valor_f * (1 - valor_f)
    perdida_datos = nn.MSELoss()(valor_f, y_analitica)

    x_limite = torch.tensor([X_BOUNDARY])
    f_limite = torch.tensor([F_BOUNDARY])
    limite = f(x_limite, params_dict) - f_limite

    perdida = nn.MSELoss()
    return perdida(interior, torch.zeros_like(interior)) + perdida(limite, torch.zeros_like(limite)) + perdida_datos

# Configuración del bucle de entrenamiento
tamaño_lote = 30
num_iter = 500
tasa_aprendizaje = 1e-1
dominio = (0.0, 10.0)  # Dominio ajustado

# Genera 2000 puntos de muestra
puntos_muestra = torch.linspace(dominio[0], dominio[1], 2000)
soluciones_analiticas = crecimiento_logistico(puntos_muestra.numpy(), R, K, P0)

# Entrena el modelo
for i in range(num_iter):
    indices = torch.randperm(puntos_muestra.size(0))[:tamaño_lote]
    x = puntos_muestra[indices]
    y_analitica = torch.tensor(soluciones_analiticas[indices], dtype=torch.float32)

    perdida = funcion_perdida(params_dict, x, y_analitica)

    grads = torch.autograd.grad(perdida, params_dict.values(), create_graph=True)
    with torch.no_grad():
        for (nombre, param), grad in zip(params_dict.items(), grads):
            param -= tasa_aprendizaje * grad
            params_dict[nombre] = param

    if i % 10 == 0:
        print(f"Iteración {i} con pérdida {float(perdida)}")


In [ ]:
import matplotlib.pyplot as plt

# Calcula la solución analítica sobre un rango de puntos para trazar
t_plot = torch.linspace(dominio[0], dominio[1], 500).numpy()
solucion_analitica_trazado = crecimiento_logistico(t_plot, R, K, P0)

# Utiliza el modelo entrenado para predecir la solución PINN sobre el mismo rango
t_tensor_trazado = torch.tensor(t_plot, dtype=torch.float32)
with torch.no_grad():
    solucion_pinn_trazado = f(t_tensor_trazado, params_dict).numpy()

# Trama los puntos de muestra, la solución analítica y las predicciones de PINN
plt.figure(figsize=(12, 6))
plt.scatter(puntos_muestra, soluciones_analiticas, color='gray', alpha=0.5, label='Puntos de Muestra (Solución Analítica)')
plt.plot(t_plot, solucion_analitica_trazado, label='Solución Analítica', color='blue')
plt.plot(t_plot, solucion_pinn_trazado, label='Predicciones de PINN', linestyle='dashed', color='red')
plt.xlabel('Tiempo')
plt.ylabel('Tamaño de la Población')
plt.title('Puntos de Muestra, Solución Analítica y Predicciones de PINN')
plt.legend()
plt.show()